In [1]:
from future.backports.datetime import datetime
%load_ext autoreload
%autoreload 2
import matplotlib
matplotlib.use('TKAgg')
from pianoq.lab.mplc.mask_aligner import MPLCAligner
from pianoq.lab.pco_camera import PCOCamera
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCMasks
from pianoq.lab.mplc.mplc_device import MPLCDevice
import os 
import datetime
import time 
dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
cam = PCOCamera()
cam.set_exposure_time(0.5)

ml = MPLCAligner(use_cam=False)
ml.update('1to5w4f', '5to11w8', pi_steps_x=[0], pi_steps_plane=1)

In [47]:
from pianoq.lab.scripts.live_camera import live_cam
roi=(750, 350, 1200, 850)
live_cam(cam, roi=roi)

In [42]:
roi=(750, 350, 1200, 850)
cam.show_image(roi=roi)

(<Figure size 640x480 with 1 Axes>, <Axes: >)

In [48]:
roi=(750, 350, 1200, 850)
name = 'U1_best.fits'
cam.save_image(path=rf'{dir_path}\{name}', roi=roi)
cam.show_image(roi=roi)

(<Figure size 640x480 with 1 Axes>, <Axes: >)

In [78]:
import glob 
from pianoq.lab.mplc.mask_utils import remove_input_modes, add_phase_input_spots
import numpy as np 
modes_to_keep = np.array([3, 8, 13, 18, 23, 28, 33, 38, 43, 48])

mplc = MPLCDevice()
mplc.restore_location()
print('Got MPLC')
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_10_14_00_34_old.masks"
masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_25_13_19_15_extra_1cm.masks"
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_17_first_diffuser\2024_09_17_12_19_58_diffuser_d1d.masks"

msks = MPLCMasks()
msks.loadfrom(masks_path)
masks = msks.real_masks
masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
phases = np.random.uniform(0, 2*np.pi, size=50)
masks = add_phase_input_spots(masks, phases)
mplc.load_masks(masks, linear_tilts=True)
mplc.restore_location()

Got MPLC


In [ ]:
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
path = rf'{dir_path}\{timestamp}_{name}.cam'
cam.save_image(path)

## motors in place

In [3]:
backlash = 0
wait_after_move = 0.3
from pianoq.lab.mplc.consts import thorlabs_x_serial, thorlabs_y_serial
from pianoq.lab.thorlabs_motor import ThorlabsKcubeDC, ThorlabsKcubeStepper
from pianoq.lab.zaber_motor import ZaberMotors
zaber_ms = ZaberMotors(backlash=backlash, wait_after_move=wait_after_move)
mxs = zaber_ms.motors[1]
mys = zaber_ms.motors[0]
print('Got Zaber motors')

mxi = ThorlabsKcubeDC(thorlabs_x_serial, backlash=backlash, wait_after_move=wait_after_move)
myi = ThorlabsKcubeStepper(thorlabs_y_serial, backlash=backlash, wait_after_move=wait_after_move)
print('Got Thorlabs motors')

Got Zaber motors
Got Thorlabs motors


In [131]:
mxi.move_absolute(9.30)
myi.move_absolute(4.05)
print(mxi.get_position())
print(myi.get_position())

9.3001
4.05


In [142]:
mxs.move_absolute(13.03)
mys.move_absolute(6.45)
print(mxs.get_position())
print(mys.get_position())

13.0300095
6.449996625


In [147]:
myi.move_relative(0.01)

In [149]:
print(mxi.get_position()) # 9.2701
print(myi.get_position()) # 3.3100

9.27007
3.3100008


BS
- best idler: ~9K
- best signal: ~8.3K 

Mirror:
- best idler: 
- best signal: 


# Small scans to see what happened 

In [12]:
# Timetagger
from pianoq.lab.time_tagger import QPTimeTagger
from pianoq.lab.mplc.consts import TIMETAGGER_DELAYS, TIMETAGGER_COIN_WINDOW
tt = QPTimeTagger(integration_time=1, remote=True,
                  single_channel_delays=TIMETAGGER_DELAYS, coin_window=TIMETAGGER_COIN_WINDOW)

### idler scan

In [130]:
from pianoq.lab.photon_scan import PhotonScanner
integration_time = 1
start_x = 8.9
end_x = 9.6
start_y = 2.7
end_y = 5

mxi.wait_after_move = 0
myi.wait_after_move = 0
# start_x = 8.5
# end_x = 9.5
# start_y = 2
# end_y = 4

resolution = 1 
x_pixels = 14 // resolution
y_pixels = 26 // resolution
pixel_size_x = 0.05 * resolution
pixel_size_y = 0.05 * resolution

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror'
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
singles_idler_path = fr'{dir_path}\{timestamp}_idler_6_spots_again.scan'

scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, is_timetagger=True, coin_window=2e-9, saveto_path=singles_idler_path)   
_ = scanner.scan(x_motor=mxi, y_motor=myi, ph=tt)

Moving to starting position...
starting scan
dur: 0. pix: 0, 0. Singles1: 3158. Singles2: 879. Coincidence: 0.00.
dur: 2. pix: 0, 1. Singles1: 3062. Singles2: 910. Coincidence: 0.00.
dur: 3. pix: 0, 2. Singles1: 2997. Singles2: 903. Coincidence: 0.00.
dur: 5. pix: 0, 3. Singles1: 2927. Singles2: 892. Coincidence: 0.00.
dur: 7. pix: 0, 4. Singles1: 3308. Singles2: 860. Coincidence: 0.00.
dur: 8. pix: 0, 5. Singles1: 3147. Singles2: 862. Coincidence: 0.00.
dur: 10. pix: 0, 6. Singles1: 3369. Singles2: 874. Coincidence: 0.00.
dur: 12. pix: 0, 7. Singles1: 3315. Singles2: 901. Coincidence: 0.00.
dur: 13. pix: 0, 8. Singles1: 3083. Singles2: 878. Coincidence: 0.00.
dur: 15. pix: 0, 9. Singles1: 3010. Singles2: 870. Coincidence: 0.00.
dur: 17. pix: 0, 10. Singles1: 3224. Singles2: 864. Coincidence: 0.00.
dur: 18. pix: 0, 11. Singles1: 3240. Singles2: 885. Coincidence: 0.00.
dur: 20. pix: 0, 12. Singles1: 3229. Singles2: 889. Coincidence: 0.00.
dur: 22. pix: 0, 13. Singles1: 3247. Singles2: 8

### signal scan

In [124]:
from pianoq.lab.photon_scan import PhotonScanner
integration_time = 1
start_x = 12.65
end_x = 13.35
start_y = 5.5
end_y = 8

mxs.wait_after_move = 0.1
mys.wait_after_move = 0.1

resolution = 1 
x_pixels = 14 // resolution
y_pixels = 50 // resolution
pixel_size_x = 0.05 * resolution
pixel_size_y = 0.05 * resolution

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror'
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
singles_signal_path = fr'{dir_path}\{timestamp}_signal_initial_spots.scan'

scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, is_timetagger=True, coin_window=2e-9, saveto_path=singles_signal_path)   
_ = scanner.scan(x_motor=mxs, y_motor=mys, ph=tt)

Moving to starting position...
starting scan
dur: 0. pix: 0, 0. Singles1: 2789. Singles2: 1091. Coincidence: 0.00.
dur: 1. pix: 0, 1. Singles1: 2742. Singles2: 1193. Coincidence: 0.00.
dur: 2. pix: 0, 2. Singles1: 2766. Singles2: 1257. Coincidence: 0.00.
dur: 4. pix: 0, 3. Singles1: 3754. Singles2: 1336. Coincidence: 0.00.
dur: 5. pix: 0, 4. Singles1: 3731. Singles2: 1471. Coincidence: 0.00.
dur: 7. pix: 0, 5. Singles1: 3620. Singles2: 1422. Coincidence: 0.00.
dur: 8. pix: 0, 6. Singles1: 3565. Singles2: 1373. Coincidence: 0.00.
dur: 9. pix: 0, 7. Singles1: 3729. Singles2: 1336. Coincidence: 0.00.
dur: 11. pix: 0, 8. Singles1: 3789. Singles2: 1440. Coincidence: 0.00.
dur: 12. pix: 0, 9. Singles1: 3741. Singles2: 1320. Coincidence: 0.00.
dur: 13. pix: 0, 10. Singles1: 3705. Singles2: 1151. Coincidence: 0.00.
dur: 15. pix: 0, 11. Singles1: 3647. Singles2: 1231. Coincidence: 0.00.
MovementFailedException: Movement may have failed because fault was observed: Stalled and Stopped (FS).
dur: 

In [98]:
from pianoq.misc.mplt import mimshow
mimshow(scanner.single1s, vmax=3700)
mimshow(scanner.single2s, vmax=1000)

(<Figure size 640x480 with 2 Axes>, <Axes: >)

# New WFM with defocus 

In [101]:
from pianoq.simulations.mplc_sim.create_wfm_masks import create_WFM_unitary_masks
from scipy.io import loadmat
all_Us_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_10_fixing_phases_different_Us\for_ronen_Haar_800.mat"
all_Us = loadmat(all_Us_path)['U1_all']
U_no = 1 
dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror'
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
masks_path =    fr'{dir_path}\U{U_no}U_{timestamp}.masks'
mplc_res_path = fr'{dir_path}\U{U_no}U_{timestamp}.mplcsim'
mplc_sim = create_WFM_unitary_masks(all_Us[:, :, U_no], N_iterations=15, out_path=masks_path)
mplc_sim.res.saveto(mplc_res_path, smaller=True)

{'wavelength': 8.1e-07, 'dist_after_plane': array([0.087 , 0.087 , 0.087 , 0.087 , 0.138 , 0.087 , 0.087 , 0.087 ,
       0.087 , 0.0954]), 'N_iterations': 15, 'Nx': 140, 'Ny': 360, 'dx': 1.25e-05, 'dy': 1.25e-05, 'max_k_constraint': 0.15, 'min_log_level': 2, 'size_factor': 3, 'use_mask_offset': True, 'symmetric_masks': False, 'active_planes': array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True]), 'N_modes': 10}


100%|██████████| 15/15 [03:07<00:00, 12.52s/it]


In [107]:
mplc_sim.res.show_overlap()

In [109]:
cam.close()